In [36]:
class Text:
    def __init__(self, text):
        text = text.split('.T\n')
        self.index = int(text[0])
        text = text[1].split('.A\n')
        self.title = text[0].replace('\n', '')
        text = text[1].split('.B\n')
        self.author = text[0].replace('\n', '')
        text = text[1].split('.W\n')
        self.bibliography = text[0].replace('\n', '')
        self.words = text[1].replace('\n', '')

In [42]:
words = []
with open('cran/cran.all.1400', 'r') as file:
    txt = file.read()
    txt = txt.split('.I')[1:]
    words = list(map(lambda x : Text(x), txt))